# PrismAI

## Data

### Upload Data to HuggingFace Hub

In [ ]:
HF_TOKEN = open("/home/mastoeck/.hf_token").read().strip()

In [ ]:
import bz2
import json
from collections import defaultdict
from pathlib import Path

from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset

In [ ]:
def flatten_samples(batch: dict):
    result = defaultdict(list)
    for element in batch["samples"]:
        for sample in element:
            for key, value in sample.items():
                result[key].append(value)
    return result


In [ ]:
from unicodedata import normalize


def normalize_text(batch: dict[str, list[str]]):
    return {
        "text": [
            " ".join(normalize("NFC", text).strip().split()) for text in batch["text"]
        ]
    }

In [ ]:
for path in sorted(Path("../data").iterdir()):
    if path.is_file() and path.stem.startswith("prismai-"):
        config_name = path.with_suffix("").stem.removeprefix("prismai-")
        print(config_name)

        raw_dataset = (
            Dataset.from_list(
                [json.loads(line) for line in bz2.open(path, "rt")],
            )
            .map(
                flatten_samples,
                batched=True,
                remove_columns=["_id", "samples"],
            )
            .map(normalize_text, batched=True)
            .map(lambda sample: {"agent": sample["agent"].replace("-", "_").replace(":", "_")})
        )
        dataset = DatasetDict(
            {
                agent: raw_dataset.filter(lambda x: x["agent"] == agent)
                for agent in set(raw_dataset["agent"])
            }
        )
        dataset.push_to_hub(
            "liberi-luminaris/PrismAI",
            config_name,
            private=True,
            token=HF_TOKEN,
        )

In [ ]:
# for agent in [
#     "gpt_4o_mini",
#     "gemma2_9b",
# ]:
datasets: list[DatasetDict] = [
    # load_dataset("liberi-luminaris/PrismAI", f"{domain}-fulltext-{agent}")
    load_dataset("liberi-luminaris/PrismAI", f"{domain}-fulltext").map(
        lambda _: {"domain": domain}
    )
    for domain in ["bundestag", "spiegel_articles", "gutenberg_de"]
]  # type: ignore
agents = set(agent for dataset in datasets for agent in dataset.keys())
dataset = DatasetDict(
    {
        agent: concatenate_datasets(
            [dataset[agent] for dataset in datasets if agent in dataset.keys()]
        )
        for agent in agents
    }
)
dataset.push_to_hub(
    "liberi-luminaris/PrismAI",
    "de-fulltext",
    # f"de-fulltext-{agent}",
    private=True,
    token=HF_TOKEN,
)

In [32]:
datasets: list[DatasetDict] = [
    load_dataset("liberi-luminaris/PrismAI", f"{domain}-fulltext").map(
        lambda _: {"domain": domain}
    )
    for domain in [
        "blog_authorship_corpus",
        "student_essays",
        "cnn_news",
        "euro_court_cases",
        "house_of_commons",
        "arxiv_papers",
        "gutenberg_en",
    ]
]  # type: ignore
agents = set(agent for dataset in datasets for agent in dataset.keys())
dataset = DatasetDict(
    {
        agent: concatenate_datasets(
            [dataset[agent] for dataset in datasets if agent in dataset.keys()]
        )
        for agent in agents
    }
)
dataset.push_to_hub(
    "liberi-luminaris/PrismAI",
    "en-fulltext",
    private=True,
    token=HF_TOKEN,
)

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Map:   0%|          | 0/42311 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/15177 [00:00<?, ? examples/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/liberi-luminaris/PrismAI/commit/d903bd37cbb12e2201d4511fe60fb3d80231bd2c', commit_message='Upload dataset', commit_description='', oid='d903bd37cbb12e2201d4511fe60fb3d80231bd2c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/liberi-luminaris/PrismAI', endpoint='https://huggingface.co', repo_type='dataset', repo_id='liberi-luminaris/PrismAI'), pr_revision=None, pr_num=None)